In [3]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 2.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678070 sha256=cab7a103953bcbb4b26c410834c27bc7bd3d84855cd835b4cd14cf2cc14351de
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [20]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [0]:
import pandas as pd
pd.options.display.max_rows=50
pd.options.display.max_columns=None
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

In [6]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [11]:
algo = SVD()
algo.fit(trainset)

In [12]:
# test는 데이터 셋 전체에 대해 추천 영화 평점 데이터를 생성
predictions = algo.test(testset)
print('prediction type : ', type(predictions), 'size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'> size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.443416890684053, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.825391096121222, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.137156397572703, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.31498437967309, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.8489199277363, details={'was_impossible': False})]

In [17]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.443416890684053),
 ('882', '291', 3.825391096121222),
 ('535', '507', 4.137156397572703)]

In [18]:
# 사용자 id, item id는 문자열로 인자로 들어감
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.22   {'was_impossible': False}


In [19]:
# 일반적으로 평가하려면 actual값과 pred값이 함께 있어야 하는데, 여기선 predictions만 있다. 이는 predictions 내부에 actual정보를 갖고 있을 수 있다는 의미
accuracy.rmse(predictions)

RMSE: 0.9490


0.9490161669725464

### OS 파일 데이터를 Surprise 데이터 셋으로 로딩

In [0]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

In [0]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader=reader)

In [0]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

In [34]:
# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 얘측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Pandas DataFrame에서 Surprise 데이터 셋으로 로딩

In [36]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

# ratings DataFrame에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지킴
daata = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Cross Validation

In [44]:
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MSE'], cv=5, verbose=True)

Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8699  0.8762  0.8797  0.8680  0.8739  0.8735  0.0042  
MSE (testset)     0.7567  0.7678  0.7738  0.7534  0.7636  0.7631  0.0074  
Fit time          4.99    5.00    5.07    5.00    5.07    5.03    0.04    
Test time         0.27    0.14    0.27    0.14    0.14    0.19    0.07    


{'fit_time': (4.990229606628418,
  5.004050970077515,
  5.071258306503296,
  4.997366905212402,
  5.066888093948364),
 'test_mse': array([0.75667916, 0.76779816, 0.77381681, 0.75342564, 0.76363682]),
 'test_rmse': array([0.86987307, 0.87624093, 0.87966858, 0.86800094, 0.87386316]),
 'test_time': (0.271756649017334,
  0.1365375518798828,
  0.27057981491088867,
  0.13792061805725098,
  0.13985943794250488)}

### Grid Search

In [47]:
# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors' : [50, 100, 200]}

# CV를 3개 폴드 셋으로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.877242873926941
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [0]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds클래스를 ratings_noh.csv 파일 기반으로 생성. 전체 데이터 셋을 학습용으로 사용할 때 그냥 불러와서 하면 에러 발생. train_test_split을 거쳐야 함
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [50]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [58]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [59]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [0]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
  
  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()
  
  # 모든 영화이ㅡ movieId 중 이미 평점을 매긴 영화이ㅡ movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수 : ', len(seen_movies), ', 추천 대상 영화 수 : ', len(unseen_movies), ', 전체 영화 수 : ', len(total_movies))
  
  return unseen_movies

In [66]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 :  46 , 추천 대상 영화 수 :  9696 , 전체 영화 수 :  9742


In [0]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  
  # 알고리즘 객체의 predict() 메서드를 평점이 엇는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
  
  def sortkey_est(pred):
    return pred.est
  
  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]
  
  # top_n으로 추출된 영화이ㅡ 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
  
  top_movie_pred = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_rating, top_movie_titles)]
  
  return top_movie_pred

In [82]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
  print(top_movie[2], ':', top_movie[1])

평점 매긴 영화 수 :  46 , 추천 대상 영화 수 :  9696 , 전체 영화 수 :  9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
